# Analyzing The Chinook Data Set 

In this project I am working with a data set of an online music store. I will use MySQL for my analysis. There are different tables in this database, which i will combine to find the answers I am looking for. Let's take a look at the structure of the database first.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT * FROM sqlite_master

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


### Most Popular Genres in the USA

The first question I want to answer is: Which are the most popular genres in the USA?
To answer this question, I am writing a query that returns each genre, with the number of tracks sold in the USA, both in absolute numbers and percentages.

In [3]:
%%sql
WITH apc AS 
(SELECT il.*, i.billing_country FROM invoice_line il 
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
 WHERE i.billing_country = 'USA'
 GROUP BY il.invoice_id)


SELECT g.name, 
ROUND(SUM(apc.quantity * apc.unit_price), 2) amount, 
ROUND(SUM(apc.unit_price*apc.quantity) / (SELECT SUM(unit_price*quantity) FROM apc)*100, 2) percentage, 
apc.billing_country 
FROM apc
INNER JOIN track t ON t.track_id = apc.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY g.name ORDER BY amount DESC





 * sqlite:///chinook.db
Done.


name,amount,percentage,billing_country
Rock,71.28,54.96,USA
Alternative & Punk,16.83,12.98,USA
Metal,9.9,7.63,USA
R&B/Soul,6.93,5.34,USA
Latin,4.95,3.82,USA
Pop,3.96,3.05,USA
Easy Listening,2.97,2.29,USA
Classical,2.97,2.29,USA
Blues,2.97,2.29,USA
Jazz,1.98,1.53,USA


The TOP 3 most popular genres for the american customer base in the Chinook online store are Rock (51 %), Metal (14 %) and Alternative & Punk (14 %).

### Analyzing Sales Agent Performance

Next, i want to analyze the performance of sales agents. I will write a query, that assigns a total dollar amount to each employee within the company.

In [4]:
%%sql 
SELECT e.first_name || " " || e.last_name name, e.title, ROUND(SUM(i.total), 2) "total amount"
FROM employee e
LEFT JOIN customer c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY name ORDER BY "total amount" DESC


 * sqlite:///chinook.db
Done.


name,title,total amount
Jane Peacock,Sales Support Agent,1731.51
Margaret Park,Sales Support Agent,1584.0
Steve Johnson,Sales Support Agent,1393.92
Robert King,IT Staff,None
Nancy Edwards,Sales Manager,None
Michael Mitchell,IT Manager,None
Laura Callahan,IT Staff,None
Andrew Adams,General Manager,None


### Market Analysis by Country

For the next step I want to analyze the different national markets. To achieve that, I group by country and calculate the total number of customers, the total and average sale value aswell as the average number of orders per customers.

To make these numbers easier to understand, I want to group countries with only one customer under the umbrella of "Other". I use a subquery with a CASE condition to achieve this. 

The result should be odered by total value of sales but I don't want "Other" to be on top of the list, since it is not a real country. I solve this by creating a hidden category, that I primarily order by while ordering by total value of sales behind that.

In [5]:
%%sql

WITH country_groups AS 
(SELECT country, 
CASE WHEN COUNT(customer_id) = 1 THEN 'Other' ELSE country END as country_or_other, 
CASE WHEN COUNT(customer_id) = 1 THEN 1 ELSE 0 END as country_group,
COUNT(customer_id) as total_customers_per_country 
FROM customer GROUP BY country ORDER BY 3)

SELECT 
cg.country_or_other country_name, 
COUNT (DISTINCT c.customer_id) total_customers, 
ROUND(SUM(i.total), 2) total_sales_value, 
ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id), 2) avg_sales_per_customer, 
ROUND(SUM(i.total) / COUNT(i.invoice_id), 2) avg_order_value
FROM customer c 
LEFT JOIN invoice i ON c.customer_id = i.customer_id
LEFT JOIN country_groups cg ON c.country = cg.country
GROUP BY 1 ORDER BY cg.country_group, 2 DESC

 * sqlite:///chinook.db
Done.


country_name,total_customers,total_sales_value,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


### Single Track Purchases vs. Album Purchases

I am interested in our customers behaviour regarding their purchasing of full albums. The background is, that the store is considering not to buy full albums anymore, if the customers are only interested in the more popular tracks and not the complete album. This would be a way to decrease the cost of the store. However, if there is a significant group of customers, who like to buy full albums, they might be dissatisified in the future.
Because there is no category for album purchase, we need to find a way to gather this information.

I wrote a query that produces a list of all unique albums first, including the number of individual tracks. Next, I wrote a query that groups by invoide_id and album_id. This basically assigns the purchases to the albums they belong to.

In [6]:
%%sql 
SELECT DISTINCT(t.album_id), a.title, COUNT(t.track_id)
FROM track t
LEFT JOIN album a ON a.album_id = t.album_id
GROUP BY a.album_id

 * sqlite:///chinook.db
Done.


album_id,title,COUNT(t.track_id)
1,For Those About To Rock We Salute You,10
2,Balls to the Wall,1
3,Restless and Wild,3
4,Let There Be Rock,8
5,Big Ones,15
6,Jagged Little Pill,13
7,Facelift,12
8,Warner 25 Anos,14
9,Plays Metallica By Four Cellos,8
10,Audioslave,14


In [7]:
%%sql 
SELECT il.invoice_id, COUNT(t.track_id) number_tracks, t.album_id 
FROM invoice_line il 
INNER JOIN track t on t.track_id = il.track_id
GROUP BY il.invoice_id, t.album_id

 * sqlite:///chinook.db
Done.


invoice_id,number_tracks,album_id
1,16,91
2,1,20
2,1,34
2,1,39
2,1,66
2,1,73
2,1,134
2,1,190
2,1,215
2,1,218


In [8]:
%%sql 
WITH purchases AS 
(SELECT il.invoice_id, COUNT(t.track_id) number_tracks, t.album_id 
FROM invoice_line il 
INNER JOIN track t on t.track_id = il.track_id
GROUP BY il.invoice_id, t.album_id),

album_list AS 
(SELECT DISTINCT(t.album_id), a.title, COUNT(t.track_id) album_tracks
FROM track t
LEFT JOIN album a ON a.album_id = t.album_id
GROUP BY a.album_id),

full_purchases AS
(SELECT p.invoice_id, al.album_id, al.album_tracks, p.number_tracks, CASE WHEN (al.album_tracks = p.number_tracks) THEN 1 ELSE 0
 END AS full_album_sale
 FROM purchases p
 LEFT JOIN album_list al ON al.album_id = p.album_id
 GROUP BY p.invoice_id)

SELECT COUNT(DISTINCT(invoice_id)) total_sales,
SUM(full_album_sale) number_full_album_sales,
COUNT(full_album_sale)-SUM(full_album_sale) track_sales,
(SUM(full_album_sale)/CAST(COUNT(invoice_id) AS float))*100 percentage_full_album_sales,
((COUNT(full_album_sale)-SUM(full_album_sale))/CAST(COUNT(invoice_id) AS float))*100 percentage_track_sales
FROM full_purchases


 * sqlite:///chinook.db
Done.


total_sales,number_full_album_sales,track_sales,percentage_full_album_sales,percentage_track_sales
614,117,497,19.055374592833875,80.94462540716613


Out of 614 sales 184 were full album sales and 430 purchases of one or more tracks but not a complete album. Almost 30% of all purchases are full album purchases. While this is not the majority of purchases, it still is a significant portion. It is also a very lucrative portion, because those purchase usually contain 10 or more tracks, while the non-album purchases are often a single track.